In [2]:
import numpy as np
import pandas as pd
import os
if not os.path.exists('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\随机占优PROMETHEE-II过程'):
    os.mkdir('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\随机占优PROMETHEE-II过程')
id2label = {k: v.strip() for k, v in enumerate(open('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\label.txt', 'r', encoding='utf-8').readlines())}
def F(k,x):
    summ=0
    for i in range(0,x+1):
        summ=summ+proportions[i][k]
    return summ
def ssd(F,k,x):
    summ=0
    for i in range(0,x+1):
        summ=summ+F(k,i)
    return summ
def tsd(F,k,x):
    summ=0
    for i in range(0,x+1):
        summ=summ+ssd(F,k,i)
    return summ
def sur(k1,h1,E):
    sumss=0
    for k in range(0,len(indexes)):
        for h in range(0,len(indexes)):
            if E[k]-E[h]>=0:
                bkh=E[k]-E[h]
            else:
                bkh=0
            if k!=h:
                sumss=sumss+bkh
    a=2*sumss/(len(indexes)*(len(indexes)-1))
    if  ss[k1,h1] in ['FSD','SSD','TSD']:
        if E[k1]>=E[h1]+a:
            return 1
        elif a+E[h1]>=E[k1]>=E[h1]:
            return np.round((E[k1]-E[h1])/a,4)
        else:
            return '--'
    else:
        return '--'
List=[]
for pinpai in ['ALBD','AND','AT','BNL','GRN','LN','QPL','TB']:
    l=[]
    indexes={'舒适':0,'耐久':0,'精细':0,'保障':0,'牌证':0,'外观':0,'外部':0,'响应及时性':0,'沟通有效性':0,'信息安全性':0,'补偿合理性':0,'服务专业性':0}

    df=pd.DataFrame(columns=[each for each in indexes],index=[1,2,3],dtype='int')
    for each in df.columns:
        for each2 in df.index:
            df[each][each2]=0
# ------------------------------------------------------------------------------------------------
    with open('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\维度-情感分类结果\维度-情感分类结果_'+pinpai+'.tsv','r',encoding='utf-8') as f:
        data2=f.read().split('\n')
    proportions=np.zeros((3,len(indexes)))
    for each in data2:
        # 对于每一条数据
        try:
            ids, probs, reviews = each.split('\t')
            if ',' in ids:
                # 复合数据
                ids = ids.split(',')
                probs = probs.split('|')
            else:
                ids = [ids]
                probs = [probs]
            for i in range(0,len(ids)):
                # 对于每一条记录
                id_for_dimension = int(ids[i]) % 12
                # 找到对应维度的id，是列
                probss = probs[i].split(',')
                # 找到对应情感的概率
                for j in range(0,len(probss)):
                    proportions[j][id_for_dimension] += float(probss[j])
        except:
            print(each)
    # 纵向归一化
    for i in range(0,len(indexes)):
        proportions[:,i]=np.round(proportions[:,i]/np.sum(proportions[:,i]),4)
# -----------------------------------------------------------------------------------------------
    pd.DataFrame(proportions).to_csv('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\随机占优PROMETHEE-II过程\A比例_'+pinpai+'.csv')
    E=np.zeros(len(indexes))
    for j in range(0,len(indexes)):
        E[j]=np.sum([proportions[i][j]*(i+1) for i in [0,1,2]])
    def suiji(k1,k2):
        dataframe=pd.DataFrame({'flag1':[],'flag2':[],'flag3':[],})
        for i in np.arange(0,3):
            dataframe.loc[i]=[F(k1,i)-F(k2,i),ssd(F,k1,i)-ssd(F,k2,i),tsd(F,k1,i)-tsd(F,k2,i)]
        if dataframe.loc[dataframe["flag1"]>0].empty:
            return 'FSD'
        elif dataframe.loc[dataframe["flag2"]>0].empty:
            return 'SSD'
        elif dataframe.loc[dataframe["flag3"]>0].empty and E[k1]>E[k2]:
            return 'TSD'
        else:
            return '--'
    ss=np.array([['None'] * len(indexes)]* len(indexes))
    for i in range(0,len(indexes)):
        for j in range(0,len(indexes)):
            ss[i,j]=suiji(i,j)
            if i==j:
                ss[i,j]='--'
    pd.DataFrame(ss).to_csv('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\随机占优PROMETHEE-II过程\B占优关系_'+pinpai+'.csv')
    SUR=np.array([['None'] * len(indexes)]* len(indexes))
    for i in range(0,len(indexes)):
        for j in range(0,len(indexes)):
            SUR[i,j]=sur(i,j,E)
    pd.DataFrame(SUR).to_csv('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\随机占优PROMETHEE-II过程\C占优程度_'+pinpai+'.csv')

    S=np.array([[0.00] * len(indexes)]*3)
    for j in range(0,len(indexes)):
        sums=0
        for each in SUR[j,:]:
            try:
                each=float(each)
                sums=sums+each
            except:
                pass
        S[0,j]=sums
    for j in range(0,len(indexes)):
        sums=0
        for each in SUR[:,j]:
            try:
                each=float(each)
                sums=sums+each
            except:
                pass
        S[1,j]=sums
    for j in range(0,len(indexes)):
        try:
            S[2,j]=float(S[0,j])-float(S[1,j])
        except:
            S[2,j]=66666
    s=np.argsort(-np.array(S[2,:]))
    S[0,:]=np.round(S[0,:],1)
    S[1,:]=np.round(S[1,:],1)
    S[2,:]=np.round(S[2,:],1)
    pd.DataFrame(S).to_csv('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\随机占优PROMETHEE-II过程\D出入流_'+pinpai+'.csv')
    l.extend(S[2,:].tolist())
    List.append(l)
pd.DataFrame(List,columns=list(indexes.keys())).to_csv('D:\python_common\代码_感知质量大论文\公开数据处理\感知质量数据集\随机占优PROMETHEE-II过程\E出入流汇总.csv',index=False,encoding='gbk')